In [1]:
!pip install transformers

In [3]:
!pip install sentencepiece

In [4]:
import numpy as np
import pandas as pd

import string
import re

import torch
from torch import nn
import random

# huggingface
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [5]:
# data logistics: load theses title and abstract
# limit_title_len=[4,10] restricts to titles in between 4 and 10 tokens
def load_thesis_data(path='../theses.tsv', limit_title_len=None):
    df = pd.read_csv(path, sep='\t')
    df = df[df['Sprache'] == 'DE']
    df['length'] = df['Titel'].apply(lambda x: len(x.split()))
    if limit_title_len != None:
        df = df[df['length'].between(limit_title_len)] 
    return df


In [6]:
# set up the models; they will download on first time use but this will take some time (1.2 GB)

tokenizer = AutoTokenizer.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")
model = AutoModelForSeq2SeqLM.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")


In [8]:

# method for summary generation, using the global model and tokenizer
def generate_summary(model, abstract, num_beams = 2, repetition_penalty = 2.5,
                    length_penalty = 2.0, early_stopping = True, max_output_length = 150):
    source_encoding=tokenizer(abstract, max_length=784, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

    generated_ids=model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams=num_beams,
        max_length=max_output_length,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        early_stopping=early_stopping,
        use_cache=True
        )

    # TODO ...map to string using tokenizer.decode and return
    preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
         for gen_id in generated_ids]

    return "".join(preds)

In [9]:
# main program
df = load_thesis_data()

# Google Colab crashed on more than 5 thesis
df = df[:5]

In [10]:
# now use the pre-trained model to generate some short summaries from the 
# abstracts, and compare them to the reference titles

# adjust these values as desired
num_beams = 2
repetition_penalty = 1.0
length_penalty = 2.0
max_output_length = 120
early_stopping = True

# sample from dataset, using abstracts as input to generate short summary (~title)
from IPython.display import HTML, display
def displaysum(summarize, generated, reference):
    display(HTML(f"""<table>
    <tr><td>summarize:</td><td>{summarize}</td></tr>
    <tr><td>generated:</td><td>{generated}</td></tr>
    <tr><td>reference:</td><td>{reference}</td></tr>
    </table>
    """))

for i in [random.randint(0, len(df) - 1) for _ in range(10)]:
    # load the values
    summarize = df.iloc[i].Abstract
    reference = df.iloc[i].Titel

    generated = generate_summary(model, summarize, num_beams, repetition_penalty, length_penalty, early_stopping, max_output_length)

    displaysum(summarize, generated, reference)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


summarize:,"Fraud und Betrüge über digitale Dienste sind für Finanzinstitute ein ernstes Problem. Im Zuge des Wandels der Bankenlandschaft werden digitale Dienste in Zukunft weiter in den Vordergrund für Banken rücken. Banken möchten hier so schnell wie möglich betrügerische Transaktionen stoppen können, um Kundenauswirkungen oder finanziellen Schäden der Bank und des Kunden entegegenzuwirken. Eine maschinelle Analyse von Transaktionsdaten ist hierbei aufgrund der Menge an Daten unerlässlich. Diese Arbeit beschäftigt sich mit der Analyse von Logdaten aus dem Online Banking der ING-DiBa AG und wie eine Frauderkennung mithilfe der Big Data Streaming APIs Apache Flink und Apache Spark ermöglicht werden könnte. Elementar ist hierbei die Implementation von Session Windows durch die genannten Applikationen. Es wird zunächst ein Überblick über nötige Kenntnisse für Big-Data-Systeme geschaffen und Fraud im Kontext der ING-DiBa AG spezifiziert, bevor ein Beweis der Umsetzungsfähigkeit der beiden Frameworks betrachtet wird. Darüber hinaus wird ein Vorschlag zur Visualisierung der Daten in Echtzeit gegeben, und vorgestellt wie ein produktiver Aufbau eines Systems aussehen könnte. Die Evaluation wurde mit einem Benchmark und verschiedener Konfigurationen von Multi-Node Clustern durchgeführt, wobei Apache Flink schneller Abschnitt als Apache Spark."
generated:,Fraud und Betrüge über digitale Dienste sind für Finanzinstitute ein ernstes Problem.
reference:,Evaluierung von Apache Flink und Apache Spark im Kontext der Echtzeitanalyse von Weblogdaten auf Fraud bei der ING-DiBa AG.


KeyboardInterrupt: ignored

In [11]:
from torch.utils.data import Dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [12]:
class ThesisDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len, max_output_len):
        self.tokenizer = tokenizer
        self.source_len = max_input_len
        self.summ_len = max_output_len
        self.Titel = df.Titel

        # T5 requires us to prepend the task
        self.Abstract = 'summarize: ' + df.Abstract

    def __len__(self):
        return len(self.Titel)

    def __getitem__(self, index):
        abstract = str(self.Abstract[index])
        title = str(self.Titel[index])

        source_tok = self.tokenizer.batch_encode_plus([abstract], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        label_tok = self.tokenizer.batch_encode_plus([title], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        input_ids = source_tok['input_ids'].squeeze()
        input_mask = source_tok['attention_mask'].squeeze()
        label_ids = label_tok['input_ids'].squeeze()
        label_mask = label_tok['attention_mask'].squeeze()

        return {
            'input_ids': input_ids.to(dtype=torch.long), 
            'input_mask': input_mask.to(dtype=torch.long), 
            'label_ids': label_ids.to(dtype=torch.long),
            'label_mask': label_mask.to(dtype=torch.long)
        }

In [14]:
# for each point in the data loader, compute the forward pass, loss and
# backward pass

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()

    for i, data in enumerate(loader, 0):
        y = data['label_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()

        # set the padding symbols to -100 to be ignored by torch
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100

        inputs = data['input_ids'].to(device, dtype=torch.long)
        mask = data['input_mask'].to(device, dtype=torch.long)

        outputs = model(input_ids = inputs, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

        loss = outputs[0]

        if i % 10 == 0:
            print({"Training Loss": loss.item()})
  
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')

In [15]:
# for validation, set the model to eval mode and compute all predictions
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
            y = data['label_ids'].to(device, dtype=torch.long)
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['input_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(input_ids=ids,
                attention_mask=mask,
                num_beams=num_beams,
                max_length=max_output_length,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                early_stopping=early_stopping,
                use_cache=True)
            
            preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]
            target = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in y]
            if i % 100 == 0:
                print(f'Completed {i}')

            predictions.extend(preds)
            actuals.extend(target)
    
    return predictions, actuals

In [16]:
# defining some parameters that will be used later on in the training  
batch_size_train = 32
batch_size_vali = 4

max_input_len = 512    # 512?
max_output_len = 120

# set random seeds and deterministic pytorch for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

if tokenizer is None:
    tokenizer = AutoTokenizer.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")

if df is None:
    df = load_thesis_data()
    
# split the dataframe into training and validation
df_train = df.sample(frac=0.8, random_state=seed)
df_vali = df.drop(df_train.index).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
print("df={df.shape}, train={df_train.shape}, vali={df_vali.shape}")

# Creating the Training and Validation dataset for further creation of Dataloader
ds_train = ThesisDataset(df_train, tokenizer, max_input_len, max_output_len)
ds_vali = ThesisDataset(df_vali, tokenizer, max_input_len, max_output_len)

# create data loaders for training and validation
from torch.utils.data import DataLoader
dl_train = DataLoader(ds_train, shuffle=True, num_workers=0, batch_size=batch_size_train)
dl_vali = DataLoader(ds_vali, shuffle=True, num_workers=0, batch_size=batch_size_vali)

df={df.shape}, train={df_train.shape}, vali={df_vali.shape}


In [17]:
# we'll start from the same ml6team/mt5-small-german-finetune-mlsum that we
# used before in our baseline experiment; we will reload it below so that we
# maintain the base model
base = model

# this time, we'll load it explicitly as a T5ForConditionalGeneration; the
# tokenizer will be the same
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [ ]:
epochs_train = 3
epochs_vali = [1, 2, 3]

models = []

for epoch in range(epochs_train):
    train(epoch=epoch, tokenizer=tokenizer, model=model, device=device, loader=dl_train, optimizer=optimizer)

    # save the model after each epoch; warning: model size is ~1.2G
    #model.save_pretrained('res/mt5-small-fine-tune-'+epoch)

    if epoch in epochs_vali:
        predictions, actuals = validate(epoch, tokenizer, model, device, dl_vali)

        # display some...
        for i in [random.randint(0, len(predictions) - 1) for _ in range(10)]:
            displaysum(None, generated, reference)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [1]:
# save last iteration
model.save_pretrained('res/mt5-small-fine-tune-theses')

NameError: ignored

In [16]:
# load model and compare outputs
base = AutoModelForSeq2SeqLM.from_pretrained("ml6team/mt5-small-german-finetune-mlsum")
fine = model  # or any other checkpoint from res/mt5-small-fine-tune-...

In [17]:
# pick some random theses (from df_vali!) and compare the two models
thesis_picks = [random.randint(0, len(df_vali) - 1) for _ in range(10)]
for num, i in enumerate(thesis_picks):
    print()

    # TODO generate a summary with each of the models
    s1 = generate_summary(base, summarize, num_beams, repetition_penalty,
                        	    length_penalty, early_stopping, max_output_length)
    s2 = generate_summary(fine, summarize, num_beams, repetition_penalty,
                        	    length_penalty, early_stopping, max_output_length)
    
    display(HTML(f"""<table>
    <tr><td>summarize:</td><td>{df_vali.iloc[i].Abstract}</td></tr>
    <tr><td>base:</td><td>{s1}</td></tr>
    <tr><td>fine:</td><td>{s2}</td></tr>
    <tr><td>reference:</td><td>{df_vali.iloc[i].Titel}</td></tr>
    </table>
    """))

summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen


summarize:,"Ziel der Masterarbeit ist es, eine mobile Anwendung zur optischen Erfassung von Zählerinformationen zu konzeptionieren und Implementieren. Dafür sollen Zählerinformationen, wie z. B. Identifikationsnummer und Zählerstand, optisch erfasst und verarbeitet werden. Des Weiteren soll es möglich sein, der Erfassungssoftware IZAR@MOBILE ein Foto vom Einbauort des entsprechenden Zählers zuzuordnen. Nachdem alle relevanten Daten aufgenommen sind, werden sie an die zentrale Erfassungs- und Auswertungssoftware IZAR@NET verschlüsselt übertragen. Dabei ist IZAR@MOBILE eine Software, die eine einfache und wirtschaftliche Auslesung von Funkzählern über mobile Endgeräte ermöglicht. Für die Verwaltung der einzelnen Zähler wird die IZAR@NET verwendet. Zuletzt soll eine Marktanalyse für verschiedene Länder über die aktuelle Verteilung von ""intelligenten Messsystemen"" und ""konventionellen Zählern"" durchgeführt werden. Zusätzlich zu der Marktanalyse soll eine Analyse über die ökonomische Verwendung der oben genannten Features durchgeführt werden."
base:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
fine:,Im Fokus dieser Arbeit steht der Einsatz von Cyber-Physischen Systemen in der Montage.
reference:,Konzeption und Implementierung einer mobilen Anwendung zur optischen Erfassung von Zählerinformationen
